In [1]:
import cv2
import numpy as np
import tqdm as t
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import skimage.feature as skf
import sklearn.neighbors as sn
from sklearn.neural_network import MLPClassifier
import os
from skimage.feature import greycomatrix, greycoprops
from skimage import io
from sklearn import preprocessing

In [2]:
def load_dataset(pathOrigin):
    "Funtion สำหรับ Load Dataset"
    
    data = []
    lable = []
    species = os.listdir(pathOrigin)
    for _class in species:
        if _class == '.' or _class == '..':
            continue
    #     print(_class)
        for _img in os.listdir(pathOrigin + _class):
            if _img == '.' or _img == '..':
                continue
            path = pathOrigin + _class + '\\' + _img
            img = cv2.imread(path)
            data.append(img)
            lable.append(_class)
    return data, lable
# load training dataset
train_data, train_label = load_dataset('C:\\Users\\akira\\Desktop\\Work\\bird-handcraft1\\archive\\train\\')
train_num = len(train_label)
# load testing dataset
test_data, test_label = load_dataset('C:\\Users\\akira\\Desktop\\Work\\bird-handcraft1\\archive\\test\\')
test_num = len(test_label)

In [3]:
def hog_hist(data):
    feature = []
    # HOG's Parameter
    winSize = (64,64)
    blockSize = (16,16)
    blockStride = (4,4)
    cellSize = (8,8)
    nbins = 16
    derivAperture = 1
    winSigma = 4.0
    histogramNormType = 0
    L2HysThreshold = 2.0000000000000001e-01
    gammaCorrection = 0
    nlevels = 64
    winStride = (8,8)
    padding = (8,8)
    locations = ((10,20),)
    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
    for i in data:
        hist = hog.compute(i,winStride,padding,locations)
        feature.append(np.array(hist))
    feature = np.array(feature)
    return feature
featureTr = hog_hist(train_data)
featureTr = featureTr.reshape(train_num, -1)
featureTs = hog_hist(test_data)
featureTs = featureTs.reshape(test_num,-1)

In [4]:
def hsv_hist(data):
    feature = []
    for i in data:
        # แปลงภาพให้อยู่บนปริภูมิสี HSV
        out = cv2.cvtColor(i, cv2.COLOR_BGR2HSV)
        # แปลงข้อมูลจากเมตริกซ์ให้อยู่ในรูปแบบเวกเตอร์ เฉพาะ Hue
        out = out[:,:,0].reshape(1,-1);
        # สร้างฮิสโตแกรมจาก Hue
        hist, bins = np.histogram(out,bins = np.arange(-0.5,256,1) )
        # Normalization เพื'อทําให้ Feature รสามารถรองรับขนาดภาพที'แตกต่างกันได้
        feature.append([hist/np.sum(hist)])
    feature = np.array(feature)
    return feature
featureTr2 = hsv_hist(train_data)
featureTr2 = featureTr.reshape(train_num, -1)
featureTs2 = hsv_hist(test_data)
featureTs2 = featureTs.reshape(test_num,-1)

In [5]:
def norm(data):
    norm_data = preprocessing.normalize(data, axis=0)
    return norm_data

In [6]:
normfeatureTr = norm(featureTr)
normfeatureTr2 = norm(featureTr2)
normfeatureTs = norm(featureTs)
normfeatureTs2 = norm(featureTs2)

In [7]:
train_data = np.concatenate((normfeatureTr, normfeatureTr2), axis=1)
test_data = np.concatenate((normfeatureTs, normfeatureTs2), axis=1)

In [8]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, C=0.01)
clf.fit(train_data, train_label)

LinearSVC(C=0.01, random_state=0)

In [9]:
predict = clf.predict(test_data)

In [12]:
print ("C =", 0.01, ",\t Accuracy:", np.mean(predict == test_label)*100, "%")

C = 0.01 ,	 Accuracy: 7.6 %


In [13]:
test = np.array([[100,10,3],
                 [200,20,1], 
                 [100, 50, 6]])
preprocessing.normalize(test, axis=0)

array([[0.40824829, 0.18257419, 0.44232587],
       [0.81649658, 0.36514837, 0.14744196],
       [0.40824829, 0.91287093, 0.88465174]])

In [17]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(train_data, train_label)
predict = neigh.predict(test_data)
print ("k =", 0.01, ",\t Accuracy:", np.mean(predict == test_label)*100, "%")

k = 0.01 ,	 Accuracy: 15.6 %


In [ ]:
predict

In [ ]:
test[0]

In [ ]:
featureTr[0]

In [ ]:
norm_fea[0]